# P&T Company
***

Para este problema trabajaremos las herramientas que nos ofrece el paquete **PuLP** de Python a traves de Jupyter Notebooks. 
Este es un ejemplo de un modelo que aplica la programcion lineal entera a un sistema de transporte entre enlatadoras y almacenes. El problema dice tal que:

## Planteamiento del Problema

 
<div style="text-align: justify">Uno de los productos más importantes de la P&T COMPANY es el chícharo enlatado. Los chicharos se preparan en tres enlatadoras (cercanas a Bellingham, Washington; Eugene, Oregon, y Albert Lea, Minnesota) y después se envían por camión a cuatro almacenes de distribución (Sacramento, California; Salt Lake City, Utah; Rapid City, South Dakota, y Albuquerque, Nuevo México) en el oeste de Estados Unidos. Debido a que los costos de embarque constituyen un gasto importante, la administración ha iniciado un estudio para reducirlos a su mínima expresión. Se ha estimado la producción de cada enlatadora durante la próxima temporada y se ha asignado a cada almacén cierta cantidad de la producción total de chícharos.
En la  sigiuiente tabla se proporciona esta información (en unidades de carga de camión) junto con el costo de transporte por camión cargado de cada combinación de enlatadora-almacén. </div>

|                | *Sacramento* | *Salt Lake* | *Rapid City* | *Alburqueque* | **Produccion** |
|:--------------:|:----------:|:---------:|:----------:|:-----------:|:--------------:|
|   *Bellingham*  |    $464 \$ $   |   $513 \$ $   |    $654\$ $   |    $867 \$ $    |      $75$      |
|     *Eugene*     |    $352 \$ $   |   $416 \$ $   |    $690 \$ $   |    $791 \$  $    |      $125$     |
|   *Albert Lea*   |    $995\$ $   |   $682 \$ $   |    $388 \$ $   |    $685 \$ $    |      $100$     |
| **Asignacion** |    $80$    |    $65$   |    $70$    |     $85$    |                |

<div style ="text-align: justify">Como se ve, hay un total de $300$ cargas de camión que se deben transportar. El problema es determinar el plan de asignación de estos embarques a las distintas combinaciones de enlatadora-almacén que minimice el costo total de transporte. </div>

## Modelo matematico
<div style ="text-align: justify"> El primer paso sera plantear nuestro modelo matematico. Empezando por definir nuestras variables de decision. Tomando en cuenta que buscamos <b>minimizar</b> los costes de transporte, y que dichos costes vienen determinados por cada camion en cada una de las distintas rutas posibles. Vemos entonces que, por cada una de las $3$ enlatadoras hay $4$ almacenes a donde puede ir la carga, cada una con un coste de transporte dependiendo de sus almacenes destinos que varia sobre la enlatadora de origen, pudiendo asi definirlas tal que: </div>

|                | *Sacramento* | *Salt Lake* | *Rapid City* | *Alburqueque* |
|:--------------:|:----------:|:---------:|:----------:|:-----------:|
|   *Bellingham*  |    $464x_{11} $   |   $513x_{12}$   |    $654x_{13} $   |    $867x_{14} $    | 
|     *Eugene*     |    $352x_{21} $   |   $416x_{22}$   |    $690x_{23} $   |    $791x_{24} $    |
|   *Albert Lea*   |    $995x_{31} $   |   $682x_{32}$   |    $388x_{33} $   |    $685x_{34} $    |  

Teniendo asi una funcion objetivo tal que: $\\ $ 
$$ \min Z = 464x_{11}+513x_{12}+654x_{13}+867x_{14}+352x_{21}+416x_{22}+690x_{23}+791x_{24}+995x_{31}+682x_{32}+388x_{33}+685x_{34}$$

Donde cada variable nos indica:
- $x_{11} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Bellingham-Sacramento.
- $x_{12} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Bellingham-Salt Lake.
- $x_{13} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Bellingham-Rapid City.
- $x_{14} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Bellingham-Alburqueque.
- $x_{21} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Eugene-Sacramento.
- $x_{22} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Eugene-Salt Lake.
- $x_{23} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Eugene-Rapid City.
- $x_{24} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Eugene-Alburqueque.
- $x_{31} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Albert Lea-Sacramento.
- $x_{32} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Albert Lea-Salt Lake.
- $x_{33} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Albert Lea-Rapid City.
- $x_{34} =$ La cantidad de camiones que en la siguiente temporada haran el recorrido Albert Lea-Alburqueque.

Cuyas restricciones sobre la produccion de cada enlatadora tenemos que son:


\begin{align*}
x_{11} + x_{12} + x_{13} + x_{14} &\geq 75 \quad &\text{(Bellingham)}\\
x_{21} + x_{22} + x_{23} + x_{24} &\geq 125 \quad &\text{(Eugene)} \\
x_{31} + x_{32} + x_{33} + x_{34} &\geq 100 \quad &\text{(Albert Lea)}\\
\end{align*}


Y cuyas restricciones sobre la asignacion de cada almacen son:


\begin{align*}
x_{11}+x_{21} +x_{31} &\geq 80 \quad &\text{(Sacramento)}\\
x_{12}+x_{22} +x_{32} &\geq 65 \quad &\text{(Salt Lake)}\\
x_{13}+x_{23} +x_{33} &\geq 75 \quad &\text{(Rapid City)}\\
x_{14}+x_{24} +x_{34} &\geq 85 \quad &\text{(Alburqueque)}\\
\end{align*}



## Aplicacion con PuLP
<div style ="text-align: justify"> Teniendo este modelo planteado pasaremos a plantear nuestro modelo dentro del esquema que ofrece Python para hallar soluciones optimas. Para ello importaremos los paquetes necesarios primero: <div>

In [ ]:
from pulp import *
import pandas as pd
import numpy as np

### Planteamiento del Modelo
Ahora que tenemos las bibliotecas preparadas, generaremos objetos que almacenen los distintos elementos de nuestro modelo:


In [ ]:
matriz_coste = np.array([[464,513,654,867],
                         [352,416,690,791],
                         [995,682,388,685]])

prod_enlat = np.array([75,125,100])

asign_alm = np.array([80,65,70,85])

#En caso de querer aplicar el mismo programa para otros problemas sera cuestion de cambiar los parametros de esta celda.
n_enlatadoras = 3
n_almacenes = 4

<div style="text-align: justify">Donde nuestra tabla de costes es descrita por una matriz $3x4$, las cuotas de produccion son almacenadas en un vector de dimension $3$ (Estamos trabajando con tres enlatadoras) y la asignacion a cada almacen corresponden a un vector de $4$ dimensiones (Estamos trabajando con cuatro almacenes). </div>

### Funcion Objetivo y Variables de decision
Ahora procederemos a generar, con los datos cargados, todo lo necesario para buscar soluciones optimas. Empezando por indicar que nuestro modelo es un problema de *minimizacion*:

In [ ]:
modelo = LpProblem("P&T_Company_gastos_transporte", LpMinimize)

Seguido de las variables de desicion:

In [ ]:
nombres_variables = [str(i)+str(j) for j in range(1,n_almacenes+1) for i in range(1, n_enlatadoras+1)]
nombres_variables.sort()
print("Indices de Variables:",nombres_variables)

<div style ="text-align: justify"> Donde tenemos los mismos indices para nuestras variables de decision que indicamos en el modelado matematico. Ahora le indicaremos a PuLP como son nuestras variables de decision haciendo uso de la funcion LpVariables.matrix:</div> 

In [ ]:
variables_decision = LpVariable.matrix("X", nombres_variables, cat = "Integer", lowBound= 0 )
transporte = np.array(variables_decision).reshape(3,4)
print("Matriz de transporte ")
print(transporte)

La funcion ```LpVariable.matrix()``` tiene $4$ argumentos en este escenario:
- El nombre de las variables "X"
- Los indices de las mismas guardados en nombres_variables
- El parametro "cat" que nos permite restringir las variables a valores enteros
- La restriccion de que todas las variables sean no negativas con el parametro "lowBound"

Teniendo esto en consideracion ahora pasaremos a construir nuestra funcion objetivo:

In [ ]:
func_obj = lpSum(transporte*matriz_coste)
#Ahora mostramos en pantalla la forma de nuestra funcion objetivo 
print(func_obj)
#Agregamos dicha funcion objetivo al modelo que tenemos.
modelo +=  func_obj

Asi, haciendo uso de las funciones disponibles en PuLP logramos establecer una funcion objetivo identica a la detallada en el modelado matematico. 

### Restricciones 

Ahora solo nos queda plantear las restricciones, las cuales son por enlatadora:

In [ ]:
for i in range(n_enlatadoras):
    print(lpSum(transporte[i][j] for j in range(n_almacenes)) >= prod_enlat[i])
    modelo += lpSum(transporte[i][j] for j in range(n_almacenes)) >= prod_enlat[i] , "Restricciones de Produccion" + str(i)


Y otras por almacen:

In [ ]:
for j in range(n_almacenes):
    print(lpSum(transporte[i][j] for i in range(n_enlatadoras)) >= asign_alm[j])
    modelo += lpSum(transporte[i][j] for i in range(n_enlatadoras)) >= asign_alm[j] , "Restricciones de asignacion " + str(j)


### Modelo en Python
<div style="text-align: justify">Para comprobar que nuestro modelo se encuentre correctamente planteado, vamos a mostrarlo por pantalla. En caso de estar bien, deberia ser identico a nuestro modelo matematico.</div>

In [ ]:
print(modelo)

<div style="text-align: justify">Como podemos observar, nuestra funcion objetivo, las restricciones sobre las variables, la produccion de cada enlatadora y la asignacion de cada almacen son identicas a nuestro modelo matematico.</div>

### Solucion
<div style="text-align: justify">Antes de conocer el valor de la funcion objetivo, tenemos que evaluar si nuestro modelo tiene una solucion optima o no, para ello hacemos: </div>

In [ ]:
modelo.solve()
print("Status:",LpStatus[modelo.status])

<div style="text-align: justify">Y viendo que la respuesta es afirmativa, podemos ver que la solucion a nuestro modelo es de:</div>

In [ ]:
for v in modelo.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print("El coste minimo es:", value(modelo.objective))

Teniendo asi una solucion donde $ Z = 152.532 $ 

Ademas, tenemos tambien que las rutas a utilizar son las siguientes:

|                | *Sacramento* | *Salt Lake* | *Rapid City* | *Alburqueque* |
|:--------------:|:----------:|:---------:|:----------:|:-----------:|
|   *Bellingham*  |    $0 $   |   $20$   |    $0 $   |    $55 $    | 
|     *Eugene*     |    $80$   |   $45$   |    $0 $   |    $0 $    |
|   *Albert Lea*   |    $0$   |   $0$   |    $70 $   |    $30 $    |  

Es decir, para la planificacion de la siguiente temporada se deben establecer las siguientes rutas:
- $20$ Rutas Bellingham-Salt Lake $\quad$ ($10.260\$ $, $7\%$ del presupuesto)
- $55$ Rutas Bellingham-Alburqueque $\quad$ ($47.685\$ $, $31\%$ del presupuesto)
- $80$ Rutas Eugene-Sacramento $\quad$ ($28.160\$ $, $19\%$ del presupuesto)
- $45$ Rutas Eugene-Salt Lake $\quad$ ($18.720\$ $, $12\%$ del presupuesto)
- $70$ Rutas Albert Lea-Rapid City $\quad$ ($27.160\$ $, $18\%$ del presupuesto)
- $30$ Rutas Albert Lea-Alburqueque $\quad$ ($20.550\$ $, $13\%$ del presupuesto)

Desde aqui podemos concluir algunos resultados:
- La ruta mas costosa en esta planificacion seria Bellingham-Alburqueque.
- La ruta de mayor transito esta siguiente temporada sera Eugene-Sacramento.
- La almacenadora de Sacramento solo recibira mercancia de la enlatadora en Eugene.
- La almacenadora en Rapid City solo recibira mercancia de la enlatadora en Albert Lea.